# Injest data & Create local DB


- Hold a couple files out (for each score)
- also get rid of our stopwords 
- move Ed Bastian file into Good lead examples - from nyt


### TODO: fix up columns to have more standardized data injestion

filename
- should match the file exactly without
group
- 
name
- what about multiple people? here's an example for a how I built this episode "Melanie Perkins and Bill Creelman"
headline

title
- altair makes this bold in tooltips by default, so I wanted to avoid using this

year
- integer

url

job_title
- (optional) easy to grab from some NYT articles

publication
- 


Delete: source, f2, untitled
also rename the group "1_data" to be earnings transcripts


In [1]:
cd ..

/Users/peterwinter/Work/IDEO/highlighter


In [2]:
# Jupyter magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# imports
import pandas as pd
from src import load_txt_files
from src import db_funcs

# Load docs & Create an index 
It's going to be a dataframe that stores:
- additional information about a document - interviewee, title, source, year, etc. 
- similarity scores
- file paths (temporary link)



get index to reflect the order of document, text, token lists

In [6]:
# grab all the files from these directories and make some tokens
input_paths = [
        "./1_data/how_i_built_this/",
#         "../1_data/example_articles/",
        "./1_data/NYT_corner_office/",
        "./1_data/good_lead_articles/",
        "./1_data/current_transformational_client_articles/",
        "./1_data/Seeking_Alpha/",
    
        ]

files, docs, paths = load_txt_files.add_files_from_dir(input_paths)
df_index_a = load_txt_files.assemble_index_files(input_paths)
df_index_b = pd.DataFrame([files, paths, docs], index=['filename', 'group', 'text']).T


a = df_index_a.set_index('filename')
b = df_index_b.set_index("filename")

df_index = b.join(a, how='left')
df_index.shape, df_index_b.shape, df_index_a.shape

58 docs found in ./1_data/how_i_built_this/
218 docs found in ./1_data/NYT_corner_office/
34 docs found in ./1_data/good_lead_articles/
13 docs found in ./1_data/current_transformational_client_articles/
501 docs found in ./1_data/Seeking_Alpha/
./1_data/how_i_built_this/ 61
./1_data/NYT_corner_office/ 218
./1_data/good_lead_articles/ 35
./1_data/current_transformational_client_articles/ 13
./1_data/Seeking_Alpha/ 53899
4 filenames with overlapping index
nan: 53400 references
conine_hibt_2020: 2 references
zimmer_hibt_2020: 2 references
john_hibt_2020: 2 references

removing duplicates from index


((824, 11), (824, 3), (824, 10))

## save everything to local database

In [7]:
# fname = './1_data/Seeking_Alpha/index.csv'

# d = pd.read_csv(fname, index_col=0)
# d.to_csv('sa_index.csv')
# d = d.drop(columns=['Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'f2'])
# d['publication'] = 'earnings call'
# d = d.set_index('filename')
# d.index = [str(i).split('.txt')[0] for i in d.index]
# d.index.name = 'filename'
# d.to_csv(fname)
# print(len(d))
# d

53899

In [12]:
db_filename = 'docs.db'
con = db_funcs.create_connection(db_filename)
df_index.to_sql('docs', con=con)

### Check overlap between actual files and index.csv contents

In [11]:
# len(df_index_a), len(df_index_b)
# files_not_in_indexed
# index_without_files

In [8]:
files_not_in_indexed = list(set(df_index_b['filename']) - set(df_index_a['filename']))
index_without_files = list( set(df_index_a['filename']) - set(df_index_b['filename']))
overlap = list( set(df_index_a['filename']) | set(df_index_b['filename']))

print(f'{len(files_not_in_indexed)} files not referenced by an index.csv')
print(f'{len(index_without_files)} indexes missing a file')
print()
print(f'{len(overlap)} overlaps')

1 files not referenced by an index.csv
1 indexes missing a file

825 overlaps


# Check if order accurate

In [13]:
index_order = df_index.index.to_list()
file_order = df_index_b['filename'].to_list()
for i in range(len(df_index)):
    filename1 = index_order[i]
    filename2 = file_order[i]
    if filename1 != filename2:
        print('ERROR', i, filename1, filename2)

In [14]:
## random spotcheck to see if name matches article
import random

def check_doc_index_match():

#     d = 'good_lead_articles'
#     subsample = df_index[df_index['group'] == d]
    subsample = df_index
    random_article = random.sample(list(subsample.index), 1)[0]
    i = df_index.index.get_loc(random_article)
    print(i, random_article)
    print(df_index.loc[random_article, 'name'])
    print('---')
    print(docs[i])
    
check_doc_index_match()

515 JOYY-Inc_Tue-Nov-17-2-24-AM
nan
---
 JOYY Inc. (NASDAQ:YY) Q3 2020 Earnings Conference Call November 16, 2020  9:00 PM ET Company Participants Jane Xie – Senior Manager-Investor Relations David Xueling Li – Chairman and Chief Executive Officer Bing Jin – Chief Financial Officer Conference Call Participants Jialong Shi – Nomura Alex Poon – Morgan Stanley Yiwen Zhang – Citi Daniel Chen – JPMorgan Lei Zhang – BofA Tian Hou – TH Capital Operator Ladies and gentlemen, thank you for standing by, and welcome to the JOYY Inc.’s Third Quarter 2020 Earnings Call. At this time, all participants are in a listen-only mode. After the management’s prepared remarks, we will have a question-and-answer session. Please note this event is being recorded. I’d now like to hand the conference over to your speaker host today, Ms. Jane Xie, the company’s Senior Investor Relations Manager. Please go ahead, ma’am. Jane Xie Thank you, operator. Good morning and good evening, everyone. Welcome to JOYY’s third 

In [10]:
# check for duplicate filenames
df_index_b[df_index_b['filename'].duplicated()]

,filename,group


# look at order mistmatches

In [18]:
missing_index = df_index_b.set_index('filename').loc[files_not_in_indexed].sort_values(by='group')
# missing_index[missing_index['dir'] == "current_transformational_client_articles"]
missing_index

,group
filename,


In [19]:
df_index_a.set_index('filename').loc[index_without_files]

,name,org,headline,year,source,url,job_title,publication,Unnamed: 0.1,date,f2,title
filename,,,,,,,,,,,,
nan,NaN,"MyoKardia, Inc.",MyoKardia's (MYOK) CEO Tassos Gianakakos on Q3...,0,NaN,https://seekingalpha.com/article/4119876-myoka...,NaN,NaN,53898.0,"Nov. 2, 2017, 8:44 PM",MyoKardia-Inc_Nov-2-2017-8-44-PM,nan (0)
